# Laboratório 03 - Precisão - Guia para a aula experimental


Nesse laboratório vamos caluclar a precisão de um sistema simples, baseado no ESP8266 e um termistor. Caso deseje trabalhar com o kit disponibilizado para vocês fora do laboratório, será necessário baixar a Arduino IDE e adicionar o suporte a placa de desenvolvimento **NodeMCU**. Um guia de instalação da placa pode ser visto na referência https://www.filipeflop.com/blog/programar-nodemcu-com-ide-arduino/

Vamos iniciar nosso notebook chamando todas as bibliotecas que usaremos. Nessa etapa, basta rodar a célula abaixo com o comando **Shift + Enter**

In [ ]:
#Caso você rode o notebook no Google Colab, é necessário primeiro instalar a biblioteca arrow e influxdb_client:
!pip install arrow influxdb_client

import numpy as np
import pandas as pd
import arrow
import requests
import re
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import matplotlib.pyplot as plt

O próximo passo é definir os endereços que serão usados para consultar e enviar os dados, o token de acesso, a organização e o bucket com os dados

In [ ]:
token = ''
user = ''
url = 'http://64.227.106.209:8086'
bucket = ''

Vamos agora conectar com o banco de dados, usando as infomções definidas anteriormente.

In [ ]:
username = ''
db_client = influxdb_client.InfluxDBClient(url=url, token=token, org=user)

## Usando o Kit de Hardware
Esse é o momento de você baixar o código dos dispositivos disponível no GitHub: https://github.com/lfgomez/INF0539/lab03/tree/main/ Você deve baixar o código **termistor**. Depois de baixar o código, abra o código do Termometro na Arduino IDE e mude os parâmetros de rede Wifi, canal de publicação (minha sugestão é usar *temperatura*) credenciais do dispositivo.

![term](https://raw.githubusercontent.com/KonkerLabs/arduino_examples/master/Termometro_MQTT/term.jpg "Termômetro")

Com o dispositivo montado, o próximo passo é compilar e gravar o Firmware. Lembre-se de mudar a board na Arduino IDE para **NodeMCU v1.0**.

## Vamos baixar os dados e ver como eles se comportam
Para iniciar esse trabalho, vamos primeiro escrever uma função que nos permita baixar os dados dos útimos "d" dias, do bucket que definimos anteriormete.

In [ ]:
def get_data(hours):
    query_api = db_client.query_api()
    stop = arrow.now().to('UTC').isoformat()[:-13]+'Z'
    start = arrow.now().shift(hours=-1*hours).to('UTC').isoformat()[:-13]+'Z'
    query = """from(bucket: \""""+bucket+"""\")
    |> range(start: """+start+""", stop: """+stop+""")
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
    |> keep(columns: ["_time", "_measurement","temperatura", "grupo"])"""
    df = query_api.query_data_frame(query, org=user)
    return df

**Agora é a hora de você tomar um café :-)**

Cheque primeiramente se os dados estão chegando no InfluxDB. Caso afirmativo, deixe o dispositivo rodando sem intervenção por uns 60 minutos. Esse tempo é necessário para estabilizar a temperatura do termistor, que, por estar na placa, vai se aquecer com o uso do microcontrolador.

Após aguardar esse tempo, vamos usar a API para pegar os dados enviados pelo dispositivo **termometro** na última hora. Caso você tenha escolhido outro canal para envio dos dados, por favor, modifique a variável **canal** na próxima célula.

Agora vamos usar a função que definimos acima e baixar a última hora de dados. Vamos aproveitar para deixar o timestamp no horário do Brasil e organizar o Dataframe pelo tempo.

In [ ]:
df = get_data(1)
df['_time'] = df['_time'].dt.tz_convert('America/Sao_Paulo')
df.index = pd.to_datetime(df['_time'])
df = df.drop(columns=['_time'])

Caso tudo tenha funcionado como esperado, você deve estar vendo seus dados na sequência, já no formato tabular do Pandas.

In [ ]:
df

Ótimo! Agora os dados estão em um formato mais fácil de ler. Mas podemos também fazer um gráfico bem simples!

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(15,8))
plt.rcParams.update({'font.size': 18})
df['temperatura'].plot()
plt.xticks(rotation=45)
plt.title('Gráfico da temperatura obtida com o termistor', size=24,y=1.02)
plt.ylabel('Temperatura [C]', size=24)
plt.xlabel('Data', size=24)
plt.grid(True)
plt.show()

Neste momento, começa a parte final desse trabalho. Vamos gerar um histograma com os valores obtidos:

In [ ]:
minimo = np.min(df['temperatura'])
maximo = np.max(df['temperatura'])
bins=np.arange(minimo-0.5,maximo+1,0.5)
hist, bin_edges = np.histogram(df['temperatura'],bins=bins)
x = (bin_edges[:-1]+bin_edges[1:])/2

Vamos visualizar o histograma calculado com os dados:

In [ ]:
plt.figure(figsize=(15,8))
plt.step(x,hist,where='mid')
plt.title('Histograma da temperatura obtida com o termistor', size=24,y=1.02)
plt.ylabel('Número de contagens', size=24)
plt.xlabel('Temperatura [C]', size=24)
plt.grid(True)
plt.show()

Vamos tentar agora fazer um ajuste gaussiano nos dados:

In [ ]:
from scipy.optimize import curve_fit

#Definindo a função que usaremos para o ajuste
def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

In [ ]:
popt,pcov = curve_fit(gaus,x,hist,p0=[1,(minimo+maximo)/2,(maximo-minimo)])

In [ ]:
plt.figure(figsize=(15,8))
plt.step(x,hist,where='mid', label='Dados')
plt.plot(np.arange(minimo-0.5,maximo+0.5,0.01), gaus(np.arange(minimo-0.5,maximo+0.5,0.01), *popt), 'r-', label='Ajuste')
plt.title('Histograma da temperatura obtida com o termistor', size=24,y=1.02)
plt.ylabel('Número de contagens', size=24)
plt.xlabel('Temperatura [C]', size=24)
plt.grid(True)
plt.legend()
plt.show()

A partir do ajuste gaussiano, podemos obter o desvio padrão dos dados. Observe que esse número é influenciado não apenas pela precisão intrínseca do sensor, mas também pela escolha do "passo" (**bin**) escolhido para o histograma.

In [ ]:
print('Valor do desvio padrão obtido via ajuste Gaussiano: ' + str((popt[2]**2)**0.5))

Usando esse valor, podemos dizer que, dentro de um nível de confiança de **3 sigmas, ou seja, aproximadamente 99%**, esse sensor tem a seguinte precisão:

In [ ]:
precision = round((3*(popt[2]**2)**0.5),1)
print('Precisão (nível de confiança de 3 sigma): ' + str(precision) + ' C')

Observe também que é possível obter o desvio padrão sem o uso de um histograma, tornando o resultado independente da escolha de "passo". Usando o resultado direto do desvio padrão, deve-se obter uma precisão um pouco melhor (embora não muito diferente) da observada na célula acima:

In [ ]:
precision = round(3*(np.std(stats_df['payload.value'])),1)
print('Precisão (nível de confiança de 3 sigma): ' + str(precision) + ' C')